In [21]:
from pathlib import Path
import polars as pl
import duckdb
from datasets import Dataset
import numpy as np
from autofaiss import build_index
from FlagEmbedding import BGEM3FlagModel
from core_pro.ultilities import make_dir, make_sync_folder, create_batch_index
from openai import OpenAI
from tqdm.auto import tqdm

In [4]:
path = make_sync_folder('dataset/item_matching')
file = path / 'data_sample_FMCG_clean.parquet'

query = f"""
select item_id
, item_name
from read_parquet('{file}')
"""
df = duckdb.sql(query).pl()
df.to_dicts()[:2]

[{'item_id': 14019452100,
  'item_name': 'Mắt Mèo Cầu Vồng Huaxi 7 Màu Tạo Hiệu Ứng 9D ( Siêu Hót )'},
 {'item_id': 25423506264,
  'item_name': '*FREESHIP* Miếng Dán Trang Trí Móng Tay Chống Thấm Nước Hoạt Hình Dễ Thương  không thấm nước cho bé'}]

In [6]:
batches = create_batch_index(df.shape[0], 100)

In [22]:
run = df["item_name"].to_list()

client = OpenAI(
    api_key="675b5f6d597b2c813acce54a4458b3a09db29dcba70022e850271af8c08cf19b",
    base_url='https://compass.llm.shopee.io/compass-api/v1',
)

lst = []
for i, val in tqdm(batches.items()):
    batch = run[val[0]:val[-1]]
    resp = client.embeddings.create(
        input=batch,
        model="compass-embedding-v3",
        dimensions=384,
    )
    lst.extend(resp.data)
    # break

  0%|          | 0/989 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [10]:
name = 'bge'
path_tmp_array = Path(path / f'tmp/array/{name}')
path_tmp_ds = Path(path / f'tmp/ds/{name}')
make_dir(path_tmp_ds)
make_dir(path_tmp_array)

file_embed = path_tmp_array / 'embed.npy'
if not file_embed.exists():
    model = BGEM3FlagModel('BAAI/bge-m3', use_fp16=True)
    embeddings = model.encode(
        df['item_name_clean'].to_list(),
        batch_size=8,
        max_length=80,
        return_dense=True,
        return_sparse=False,
        return_colbert_vecs=False
    )['dense_vecs']
    np.save(file_embed, embeddings.astype(np.float64))
else:
    embeddings = np.load(file_embed)
print(embeddings.shape)

Fetching 30 files:   0%|          | 0/30 [00:00<?, ?it/s]

pre tokenize: 100%|██████████| 12341/12341 [00:01<00:00, 6375.40it/s]
You're using a XLMRobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
Inference Embeddings: 100%|██████████| 12341/12341 [01:07<00:00, 181.72it/s]


(98722, 1024)


In [8]:
embeddings.astype(np.float64).dtype

dtype('float64')

In [4]:
df = df.with_columns(pl.Series(values=embeddings, name='embed'))
dataset = Dataset.from_polars(df)
dataset.set_format(type='numpy', columns=['embed'], output_all_columns=True)

In [5]:
path_index = Path(path / 'tmp/index')
build_index(
    embeddings=embeddings,
    index_path=str(path_index / f'ip.index'),
    index_infos_path=str(path_index / f'index.json'),
    save_on_disk=True,
    metric_type='ip',
    verbose=30,
)

  0%|          | 0/1 [00:00<?, ?it/s]


(<faiss.swigfaiss_avx2.IndexHNSWFlat; proxy of <Swig Object of type 'faiss::IndexHNSWFlat *' at 0x7b8858053ea0> >,
 {'index_key': 'HNSW15',
  'index_param': 'efSearch=5226',
  'index_path': '/home/kevin/Downloads/Item_Matching_Test/tmp/index/ip.index',
  'size in bytes': 42050218,
  'avg_search_speed_ms': 9.97823628438878,
  '99p_search_speed_ms': 10.557658101897687,
  'reconstruction error %': 0.0,
  'nb vectors': 9936,
  'vectors dimension': 1024,
  'compression ratio': 0.9678393581693203})

In [6]:
# add index
dataset.load_faiss_index('embed', path_index / f'ip.index')

In [7]:
score, result = dataset.get_nearest_examples_batch(
    'embed',
    np.asarray(dataset['embed']),
    k=5
)

dict_ = {'score': [list(i) for i in score]}
df_score = pl.DataFrame(dict_)
df_result = (
    pl.DataFrame(result).drop(['embed'])
    .select(pl.all().name.prefix(f'db_'))
)

In [8]:
df_match = pl.concat([df, df_result, df_score], how='horizontal')
col_explode = [i for i in df_match.columns if 'notebooks' in i] + ['score']

In [10]:
path_export = path / 'text_match'
make_dir(path_export)
df_match.write_parquet(path_export / f'{name}.parquet')